In [ ]:
!pip3 install ipynb --upgrade

In [ ]:
import import_ipynb
from NER import get_entities
from sentiment_analysis import predictSentiment
import pandas as pd 
import datetime
import requests
from collections import Counter
import glob

In [3]:
predictSentiment("i love this")

1

In [4]:
get_entities('this GME is making me rich')

['GME']

### Load csv

In [22]:
#df_clean = pd.read_csv('data_files/wallstreetbets_21.csv', lineterminator='\n')
data = pd.read_csv('../Data_Collecting/data_files/Stocks_34.csv')
data

,Unnamed: 0,comment_id,created,fullname,parent_id,subreddit,text,post_id,title
0,0,gzsuulr,2021-05-28 20:39:11,t1_gzsuulr,t1_gzpc2r8,stocks,They denied me when I asked for a BRK discount :(,NaN,NaN
1,1,NaN,2021-05-28 20:39:14,t3_nn724o,NaN,stocks,I'm sure you know those two has stock splits. ...,nn724o,TTD or NVDA which one is better
2,2,gzsuwbn,2021-05-28 20:39:34,t1_gzsuwbn,t3_nn5tcm,stocks,Mom?,NaN,NaN
3,3,gzsuyhy,2021-05-28 20:40:03,t1_gzsuyhy,t1_gzrz5t7,stocks,I have medical conditions that they are workin...,NaN,NaN
4,4,gzsuz0n,2021-05-28 20:40:10,t1_gzsuz0n,t1_gzssh7x,stocks,It's daunting at first. A quick list of topics...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
996,996,gzv1em4,2021-05-29 11:30:53,t1_gzv1em4,t3_nn724o,stocks,You missed the big run up on TTD after its ear...,NaN,NaN
997,997,gzv1epl,2021-05-29 11:30:55,t1_gzv1epl,t1_gzu5fmn,stocks,I’ve been investing since summer of 18. That’s...,NaN,NaN
998,998,gzv1g5o,2021-05-29 11:31:29,t1_gzv1g5o,t1_gzv12o4,stocks,I'm not investing due to their ethics. They kn...,NaN,NaN
999,999,gzv1kwz,2021-05-29 11:33:23,t1_gzv1kwz,t3_nnl9c5,stocks,I will also be looking to trade it around 200....,NaN,NaN


In [24]:
d = pd.read_csv('../Data_Collecting/data_files/Stocks_3.csv')
d

,Unnamed: 0,comment_id,created,fullname,parent_id,subreddit,text,post_id,title
0,0,gylrunc,2021-05-18 19:10:20,t1_gylrunc,t3_nf66zs,stocks,And it’s all going green again,NaN,NaN
1,1,gylrwn7,2021-05-18 19:10:43,t1_gylrwn7,t1_gylh7q1,stocks,Uh no. I already signed up forv$10K add on to ...,NaN,NaN
2,2,gylrzsa,2021-05-18 19:11:19,t1_gylrzsa,t1_gylhiir,stocks,Billionaire investors aren’t actually any smar...,NaN,NaN
3,3,gylrzv8,2021-05-18 19:11:20,t1_gylrzv8,t1_gylhiir,stocks,"Well, he’s a monkey, so...",NaN,NaN
4,4,gyls2uk,2021-05-18 19:11:54,t1_gyls2uk,t3_nfj008,stocks,There are only 2 rules i follow in investing:\...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
996,996,gyn37rf,2021-05-19 01:06:16,t1_gyn37rf,t1_gyn2tuk,stocks,You said they wouldn’t do shit with a small am...,NaN,NaN
997,997,gyn3cx3,2021-05-19 01:07:29,t1_gyn3cx3,t3_nfrmgh,stocks,Bruh let me tell you the bags I’m still holdin...,NaN,NaN
998,998,gyn3d63,2021-05-19 01:07:32,t1_gyn3d63,t1_gymwjcc,stocks,I actually remember people showing higher buy ...,NaN,NaN
999,999,gyn3e48,2021-05-19 01:07:46,t1_gyn3e48,t3_nfrmgh,stocks,Evfm $2.00 may 21 call. Tanked 34% ATH yesterd...,NaN,NaN


In [30]:
old_filenames = []
old_filenames.append('Data_Collecting/data_files/wallstreetbets_50.csv')

In [31]:
old_filenames.append('Data_Collecting/data_files/wallstreetbets_44.csv')

In [ ]:
# read all csv files
for file_name in glob.glob('../Data_Collecting/data_files/'+'*.csv'):
    print('* '+file_name)
    if file_name not in old_filenames:
        data = pd.read_csv(file_name,lineterminator='\n')
        # Transform data
        print('   * Transforming data')
        dfs = transform_reddit_data(data)
        df_submissions = dfs[0]
        df_comments = dfs[1]
        # Persist submissions and comments 
        print('   * Persisting submissions')
        insert_submissions(df_submissions)
        print('   * Persisting comments')
        insert_comments(df_comments)
        old_filenames.append(file_name)

* ../Data_Collecting/data_files/Stocks_3.csv
* ../Data_Collecting/data_files/wallstreetbets_52.csv
* ../Data_Collecting/data_files/wallstreetbets_46.csv
* ../Data_Collecting/data_files/wallstreetbets_5.csv
* ../Data_Collecting/data_files/Stocks_15.csv
* ../Data_Collecting/data_files/Stocks_29.csv
* ../Data_Collecting/data_files/Stocks_28.csv
* ../Data_Collecting/data_files/Stocks_14.csv
* ../Data_Collecting/data_files/wallstreetbets_4.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* ../Data_Collecting/data_files/wallstreetbets_47.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* ../Data_Collecting/data_files/wallstreetbets_53.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments


# TESTING

In [ ]:
data = pd.read_csv('../Data_Collecting/data_files/Stocks_3.csv')
df_submissions = data.drop(['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title'],axis=1)
df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
df_submissions = df_submissions.dropna()
df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
# Apply NER
df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
# Apply Sentiment Analysis
df_submissions['Sentiment'] = df_submissions['text'].apply(predictSentiment)
insert_submissions(df_submissions)
df_comments = data.drop(['Unnamed: 0','post_id', 'fullname', 'title'],axis=1)
df_comments['created'] = df_comments['created'].apply(datetime_to_date)
df_comments = df_comments.dropna()
# Apply NER
df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
# Apply Sentiment Analysis
df_comments['Sentiment'] = df_comments['text'].apply(predictSentiment)
insert_comments(df_comments)


In [38]:
df_comments[5:55]
#df_comments.to_csv('comments_sample_data.csv')

,comment_id,created,parent_id,subreddit,text,Organizations
5,gylcu2y,2021-05-18,0 t1_gyj1f0z dtype: string,wallstreetbets,I'll take odds on Burry against Musk any day o...,[]
6,gylcu3q,2021-05-18,0 t3_nf6oeg dtype: string,wallstreetbets,Okay BB 👀,[]
7,gylcu68,2021-05-18,0 t3_neq7g4 dtype: string,wallstreetbets,Gonna try and get all the AMC I can get my han...,[AMC]
8,gylcub5,2021-05-18,0 t1_gyjbz75 dtype: string,wallstreetbets,I called it!,[]
9,gylcuh2,2021-05-18,0 t3_nf6oeg dtype: string,wallstreetbets,I need to stop paper handing,[]
10,gylcuiz,2021-05-18,0 t3_nf6oeg dtype: string,wallstreetbets,GSAT give me everything you got!,[]
11,gylcujf,2021-05-18,0 t1_gykzja9 dtype: string,wallstreetbets,"That’s the end goal, defeat the hedgies",[]
12,gylcun8,2021-05-18,0 t1_gykfpr0 dtype: string,wallstreetbets,"Hey bud, do you know how a lease works? Land o...",[]
13,gylcupk,2021-05-18,0 t3_nf8opv dtype: string,wallstreetbets,Are we all still under the impression that the...,[]
14,gylcusa,2021-05-18,0 t1_gyh2jr9 dtype: string,wallstreetbets,"I reckon they keep a museum of screenshots, li...",[]


----

## Transforming the data

In [9]:
# Returns list with submissions dataframe and comments dataframe
def transform_reddit_data(data: pd.DataFrame):
        dfs = []
        # submissions
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_submissions = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_submissions = data.drop(['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title'],axis=1)
        df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
        df_submissions = df_submissions.dropna()
        df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
        df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_submissions)
        # comments
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_comments = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_comments = data.drop(['Unnamed: 0','post_id', 'fullname', 'title'],axis=1)
        df_comments['created'] = df_comments['created'].apply(datetime_to_date)
        df_comments = df_comments.dropna()
        df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_comments)
        
        return dfs

In [10]:
# Changes utc timestamp to datetime.date
def datetime_to_date(timestamp):
    return pd.to_datetime(timestamp).date()

In [11]:
# Mentions that we are interested in
selected_orgs =  ['HKG', 'Alibaba','AMC', 'Palantir Technologies', 'PLTR', 'FORD', 'Lordstown Motors', 'RIDE', 'Virgin Galactic', 'SPCE', 'AI', 'C3.AI', 'TSLA', 'GE', 'GME', 'AAPL', 'Tesla', 'Apple', 'General Electric', 'GE', 'NOK', 'Nokia']
orgs_dict = {"Alibaba":"HKG","AMC":"AMC","Palantir Technologies":"PLTR","FORD":"FORD", "Lordstown Motors":"RIDE","Virgin Galactic":"SPCE","c3.AI":"AI","Tesla":"TSLA", "General Electric":"GE","Apple":"AAPL","GameStop":"GME","Gamestop":"GME", "Nokia":"NOK"}


Clean orgs

In [13]:
# returning a list of mentioned tickers
def clean_orgs(organizations):
    orgs = []
    for org in organizations:
        if org in selected_orgs:
            if org in orgs_dict:
                org = orgs_dict[org]
                orgs.append(org)
            else: 
                orgs.append(org)
    for org in orgs:
        o = set(orgs)
        orgs = list(o)
    return orgs

In [14]:
clean_orgs(['Virgin Galactic'])

['SPCE']

## A Look at the mentions

In [ ]:
# Creates a list of mentionend entities

#data['Organizations'] = data['text'].apply(get_entities)
#orgs = data['Organizations'].to_list()
#orgs_flat = [org for sublist in orgs for org in sublist] # Pulls out entities from the nested lists in orgs => new flat list
# Print 20 most mentions ORGs
#from collections import Counter
#org_freq = Counter(orgs_flat)
#org_freq.most_common(20)                                                       

## Transform and save the data to database

In [ ]:
#submissionsransforming into lists of submissions and comments
dfs = transform_reddit_data(data)
submissions = dfs[0]
comments = dfs[1]

In [ ]:
## SUBMISSIONS subset
#print(len(submissions))
df = submissions[1:500]
insert_submissions(df)
#COMMENTS subset
#print(len(comments))
#df_c = comments[500:5500]
#
#insert_comments(df_c)

#insert_c(df_c)

In [28]:
url = 'http://localhost:5050/'

In [15]:
def insert_submissions(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        #create submission nodes
        url2 = 'submission/'+row['subreddit']+'/'+row['created'].strftime('%Y-%m-%d')+'/'+row['id']
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('submission id '+ row['id'] + ' failed insertion')
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/submission/'+row['id']+'/'+ org + '/' + row['created'].strftime('%Y-%m-%d')
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mentions between '+ row['id'] + ' and ' + org + ' failed creation')

In [16]:
def insert_comments(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        
        #create comment nodes
        url2 = 'comment/'+row['subreddit']+'/'+row['created'].strftime('%m-%d-%Y')+'/'+row['comment_id']+'/'+row['parent_id']
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('comment id '+ row['comment_id'] + ' failed insertion')
            
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/comment/'+row['comment_id']+'/'+ org +'/'+ row['created'].strftime('%m-%d-%Y')
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mention between '+ row['comment_id'] + ' and ' + org + ' failed creation')
    
           
        # create replies relationships
        row['parent_id'] = pd.Series(row['parent_id'], dtype="string") #turning parent_id from type Series to String
        for parent_id in row['parent_id']:
            p_id = parent_id[3:]
            if parent_id[:2] == 't1':
                url_reply = 'connection/replies/comment/' + row['comment_id'] + '/' + p_id
            elif parent_id[:2] == 't3':
                url_reply = 'connection/replies/submission/' + row['comment_id'] + '/' + p_id
            response = requests.post(url+url_reply)
            if not response.status_code == 200:
                print ('replies relationship between '+ row['comment_id'] + ' and ' + p_id + ' failed creation')